Заполните данные о необходимых переменных:

In [56]:
Today = '29/02/24'
Mounth = 'Feb'

Group = 'SUEK'
# Group = 'EUROCHEM'

In [20]:
import pandas as pd

In [2]:
from datetime import date
Output_file = "_".join([str(date.today()), Group, 'SAP', 'Debt', f'{Mounth}.xlsx'])
Sheet_in_output_file = 'Debt'
Output_file

'2024-03-26_SUEK_SAP_Debt_Feb.xlsx'

Открываю следующие файлы:

In [3]:
import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\OCP")
from Defs import merge_SalesUnits
from Defs import merge_Mapping
from Defs import Period
from Defs import new_list
from Defs import export_from_RISKCUSTOM
from Defs import add_in_currency_column

In [15]:
query = f"""SELECT * FROM "RISKACCESS"."sapPositionArrears" WHERE "reportDate" = TO_DATE('{Today}', 'DD/MM/YY') AND "productType" in (130,131,132,133)"""
data_export = export_from_RISKCUSTOM(query)

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:78: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


In [57]:
import numpy as np

data_Debt = data_export.loc[data_export['productTypeName'].isin(['Транш по займам', 'Транш по кредитам'])].reset_index(drop=True)

data_Debt = data_Debt.groupby(['companyName', 'relationshipPartner',
       'businessPartnerName', 'positionCurrency', 'accountAssignmentLinkName', 'termEnd']).agg({'purchaseValueDisplayCurrency' : sum,
                                                                          'purchaseValuePositionCurrency' : sum}).reset_index()

data_Debt['purchaseValuePositionCurrency'] = data_Debt['purchaseValuePositionCurrency'] * np.sign(data_Debt['purchaseValueDisplayCurrency'])
data_Debt = add_in_currency_column(data_Debt, col_with_CCY='positionCurrency', col_with_VAL='purchaseValuePositionCurrency', day_for_export="29/02/24", CCY_to='USD')

data_Debt = data_Debt.rename(columns={"businessPartnerName": "Counterparty", "companyName": "Entity", "positionCurrency": "Currency", "purchaseValuePositionCurrency": "amoutn_outstanding", "in_USD": "amount_USD_eq", 'purchaseValuePositionCurrency_in_USD':'amount_USD_eq'}).reset_index(drop=True)

data_Debt['Entity_code'] = merge_Mapping(data_Debt, col='Entity')
data_Debt['Entity_group'] = merge_SalesUnits(data_Debt, col='Entity_code', merge_col='ocpSegment')
data_Debt['Counterparty_code'] = merge_Mapping(data_Debt, col='Counterparty')
data_Debt['Counterparty_Group'] = merge_SalesUnits(data_Debt, col='Counterparty_code', merge_col='ocpSegment')

data_Debt = Period(data_Debt, day_for_count=Today, col_with_date='termEnd')

data_Debt['Holding'] = merge_SalesUnits(data_Debt, col='Entity_code', merge_col='holding')
data_Debt = data_Debt.query('Holding == @Group').reset_index(drop=True)

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_1652\1325817245.py:6: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  'businessPartnerName', 'positionCurrency', 'accountAssignmentLinkName', 'termEnd']).agg({'purchaseValueDisplayCurrency' : sum,
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_1652\1325817245.py:6: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  'businessPartnerName', 'positionCurrency', 'accountAssignmentLinkName', 'termEnd']).agg({'purchaseValueDisplayCurrency' : sum,
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:78: UserWarning: pandas only supports SQLAlchemy connectable (engine/conn

In [17]:

manual1 = data_Debt.loc[data_Debt.Entity_code == 'External' ,['Entity', 'Entity_code']].drop_duplicates()
manual2 = data_Debt.loc[data_Debt.Counterparty_code == 'External' ,['Counterparty', 'Counterparty_Group']].drop_duplicates()
manual1

,Entity,Entity_code


Записываю данные в output file:

In [18]:
new_columns = ['dealClass1','dealClass2','instrumentOwner','dealSet','facility','Source']
data_Debt = data_Debt.reindex(columns=(data_Debt.columns.tolist() + new_columns))
data_Debt['Source'] = 'SAP'

data = data_Debt[['Entity', 'Entity_group', 'Counterparty', 'Counterparty_Group', 'amoutn_outstanding', 'Currency', 'amount_USD_eq', 'termEnd', 'Days', 'Period', 'dealClass1','dealClass2','instrumentOwner','dealSet','facility','Source']]

In [12]:
data.head(2)

,Entity,Entity_group,Counterparty,Counterparty_Group,amoutn_outstanding,Currency,amount_USD_eq,termEnd,Days,Period,dealClass1,dealClass2,instrumentOwner,dealSet,facility,Source
0,Black Sand Commodities,SUEK INT,AIM Capital Ltd,External,2.524600e+07,USD,2.524600e+07,2026-09-15,929.0,2Y+,NaN,NaN,NaN,NaN,NaN,SAP
1,Black Sand Commodities,SUEK INT,Erglis Limited,External,1.302614e+09,RUB,1.428303e+07,2029-10-08,2048.0,2Y+,NaN,NaN,NaN,NaN,NaN,SAP


In [13]:
data.to_excel(Output_file, sheet_name = Sheet_in_output_file, index = False)

Проверка на наличие тех же записей в Quantum:

In [21]:
quantum_data_export = pd.read_excel('C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\OCP\\Quantum\\2024-03-21_EuroChem_quantum_Debt_Feb.xlsx')
quantum_data_export.head(2)

,entity,Entity_group,counterparty,Counterparty_Group,amoutn outstanding,Currency,amount USD eq,termEnd,Days,Period,dealClass1,dealClass2,instrumentOwner,dealSet,facility,Source
0,BRHRG,SAMSALES,Morgan Stanley BR,External,-8011.33,BRL,-1616.101732,2031-12-31,2862,2Y+,Loan,MM,EC_BORROWINGS,-,HRG Judicial recovery BRL,Quantum
1,BRHRG,SAMSALES,Morgan Stanley BR,External,-9958.01,BRL,-2008.799688,2032-12-31,3228,2Y+,Loan,MM,EC_BORROWINGS,-,HRG Judicial recovery BRL,Quantum


In [22]:
quantum_data_export['E_Cgroup'] = quantum_data_export.entity + quantum_data_export.Counterparty_Group + quantum_data_export.Currency + quantum_data_export.termEnd.astype(str) + quantum_data_export['amoutn outstanding'].abs().astype(str)
data_Debt['E_Cgroup'] = data_Debt.Entity_code + data_Debt.Counterparty_Group + data_Debt.Currency + data_Debt.termEnd.astype(str) + data_Debt.amoutn_outstanding.abs().astype(str)

iner_list = list(set(data_Debt['E_Cgroup']).intersection(set(quantum_data_export.E_Cgroup)))

SAP_iner = data_Debt[data_Debt.E_Cgroup.isin(iner_list)]
Quantum_iner = quantum_data_export[quantum_data_export.E_Cgroup.isin(iner_list)]

In [23]:
iner_list_2 = SAP_iner.index.tolist()

SAP_iner = data_Debt[data_Debt.index.isin(iner_list_2)]
data_Debt_not_iner = data_Debt[~data_Debt.index.isin(iner_list_2)]

SAP_iner.shape, Quantum_iner.shape

((16, 23), (16, 17))

In [24]:
SAP_iner = SAP_iner[['Entity', 'Entity_group', 'Counterparty', 'Counterparty_Group', 'amoutn_outstanding', 'Currency', 'amount_USD_eq', 'termEnd', 'Days', 'Period', 'dealClass1','dealClass2','instrumentOwner','dealSet','facility','Source']]
data_Debt_not_iner = data_Debt_not_iner[['Entity', 'Entity_group', 'Counterparty', 'Counterparty_Group', 'amoutn_outstanding', 'Currency', 'amount_USD_eq', 'termEnd', 'Days', 'Period', 'dealClass1','dealClass2','instrumentOwner','dealSet','facility','Source']]


In [25]:
data_Debt_not_iner.to_excel(Output_file, sheet_name=Sheet_in_output_file, index=False)
new_list(SAP_iner, Output_file, 'Inersection')
new_list(quantum_data_export, Output_file, 'Debt_Quantum')

Проверка встречных требований:

In [58]:
if_list = ["Краткосрочные займы полученные - cash pooling", "Краткосрочные займы выданные - cash pooling"]
data_E_to_C = data_Debt[data_Debt.accountAssignmentLinkName.isin(if_list)].reset_index(drop=True)
data_E_to_C.head(2)

,Entity,relationshipPartner,Counterparty,Currency,accountAssignmentLinkName,termEnd,purchaseValueDisplayCurrency,amoutn_outstanding,Coef_to_USD,amount_USD_eq,Entity_code,Entity_group,Counterparty_code,Counterparty_Group,Days,Period,Holding
0,"АО ""Абаканская ТЭЦ""",Холдинг,"ООО ""Сибирская генерирующая компания""",RUB,Краткосрочные займы полученные - cash pooling,2031-12-22,-1.687461e+09,-1.687461e+09,0.010965,-1.850285e+07,VH00,SGC,1R00,SGC,2853.0,2Y+,SUEK
1,"АО ""Дальтрансуголь""",Холдинг,"АО ""СУЭК""",RUB,Краткосрочные займы выданные - cash pooling,2030-03-22,1.254782e+10,1.254782e+10,0.010965,1.375856e+08,8200,NTC,1100,SUEK RU,2213.0,2Y+,SUEK


In [59]:
data_received = data_E_to_C.sort_values(by=['accountAssignmentLinkName', 'Entity_code', 'Counterparty_code', 'Days']).query('accountAssignmentLinkName == @if_list[0]')
data_issued = data_E_to_C.sort_values(by=['accountAssignmentLinkName',  'Counterparty_code', 'Entity_code', 'Days']).query('accountAssignmentLinkName == @if_list[1]')

In [60]:
data_received = data_received[['Entity', 'Counterparty', 'Currency', 'amoutn_outstanding', 'termEnd']]
data_issued = data_issued[['Entity', 'Counterparty', 'Currency', 'amoutn_outstanding', 'termEnd']]
pd.concat([data_received, data_issued], axis=1).to_excel('manual.xlsx', index=False)

In [65]:
query = f"""SELECT * FROM "RISKACCESS"."sapPositionArrears" WHERE ("reportDate" BETWEEN TO_DATE('31/12/23', 'DD/MM/YY') AND TO_DATE('25/03/24', 'DD/MM/YY'))
AND 
"productType" in (130,131,132,133)"""
data_export = export_from_RISKCUSTOM(query)

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:78: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


In [82]:

if_list_2 = ['АО "Ковдорский ГОК"',  'АО "МХК "ЕвроХим"']
data_check = data_export[data_export.partnerCharacteristic == 'ЮКБ']
data_check = data_check[['companyName', 'businessPartnerName', 'positionCurrency', 'accountAssignmentLinkName', 'termEnd', 'reportDate','purchaseValueDisplayCurrency', 'purchaseValuePositionCurrency']]

data_check = data_check[data_check.companyName.str.contains('ГОК') | data_check.businessPartnerName.str.contains('ГОК')]
data_check = data_check[data_check.companyName.str.contains('МХК') | data_check.businessPartnerName.str.contains('МХК')]


In [83]:
data_check.to_excel('MHC_to_GOC.xlsx', index=False)

In [87]:
data_group = data_check.groupby(['companyName','businessPartnerName','positionCurrency','accountAssignmentLinkName','termEnd', 'reportDate'], as_index=False).agg({'purchaseValuePositionCurrency': sum})
data_group

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_1652\689942149.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  data_group = data_check.groupby(['companyName','businessPartnerName','positionCurrency','accountAssignmentLinkName','termEnd', 'reportDate'], as_index=False).agg({'purchaseValuePositionCurrency': sum})


,companyName,businessPartnerName,positionCurrency,accountAssignmentLinkName,termEnd,reportDate,purchaseValuePositionCurrency
0,"АО ""Ковдорский ГОК""","АО ""МХК ""ЕвроХим""",RUB,Краткосрочные займы выданные - cash pooling,2024-06-30,2024-01-25,1.318714e+09
1,"АО ""Ковдорский ГОК""","АО ""МХК ""ЕвроХим""",RUB,Краткосрочные займы выданные - cash pooling,2024-06-30,2024-01-26,1.373596e+09
2,"АО ""Ковдорский ГОК""","АО ""МХК ""ЕвроХим""",RUB,Краткосрочные займы выданные - cash pooling,2024-06-30,2024-01-27,1.373596e+09
3,"АО ""Ковдорский ГОК""","АО ""МХК ""ЕвроХим""",RUB,Краткосрочные займы выданные - cash pooling,2024-06-30,2024-01-28,1.373596e+09
4,"АО ""Ковдорский ГОК""","АО ""МХК ""ЕвроХим""",RUB,Краткосрочные займы выданные - cash pooling,2024-06-30,2024-01-29,1.373596e+09
...,...,...,...,...,...,...,...
147,"АО ""МХК ""ЕВРОХИМ""","АО ""Ковдорский ГОК""",RUB,Краткосрочные займы полученные - cash pooling,2024-06-30,2024-01-27,1.373596e+09
148,"АО ""МХК ""ЕВРОХИМ""","АО ""Ковдорский ГОК""",RUB,Краткосрочные займы полученные - cash pooling,2024-06-30,2024-01-28,1.373596e+09
149,"АО ""МХК ""ЕВРОХИМ""","АО ""Ковдорский ГОК""",RUB,Краткосрочные займы полученные - cash pooling,2024-06-30,2024-01-29,1.373596e+09
150,"АО ""МХК ""ЕВРОХИМ""","АО ""Ковдорский ГОК""",RUB,Краткосрочные займы полученные - cash pooling,2024-06-30,2024-01-30,1.482539e+09


In [93]:
data_GOC = data_group[data_group.companyName =='АО "Ковдорский ГОК"'].sort_values(by=['accountAssignmentLinkName', 'termEnd', 'reportDate'])
data_MHC = data_group[data_group.businessPartnerName =='АО "Ковдорский ГОК"'].sort_values(by=['accountAssignmentLinkName', 'termEnd', 'reportDate'])
new_list(pd.concat([data_GOC, data_MHC], axis=1),'MHC_to_GOC.xlsx', 'grupby_2')


In [85]:
new_list(data_group, 'MHC_to_GOC.xlsx', 'grupby', index=True)